***1)Import libraries, upload dataset and visualize data using basic commands***

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

In [ ]:
dataset = pd.read_csv('/content/Training Data.csv')
#testset = pd.read_csv('/content/Test Data.csv')
print(dataset)
ds=pd.DataFrame()

In [ ]:
dataset.shape

In [ ]:
k=pd.DataFrame(dataset)
k.dtypes

In [ ]:
dataset.isnull().any()

In [ ]:
dataset.info()

***2)ENCODING***

In [ ]:
ds=dataset.copy()
ds=ds.drop(["car_ownership","married","house_ownership","current_job_years","current_house_years"],axis=1)
ds
#print(dataset)

In [ ]:
'''from sklearn.preprocessing import LabelEncoder
#single=1,married=0
le=LabelEncoder()
ds["married"]=le.fit_transform(dataset["married"])
print(ds["married"])'''

In [ ]:
'''from sklearn.preprocessing import LabelEncoder
#norent_noown=0 owned=1 rented=2
le_1=LabelEncoder()
ds["house_ownership"]=le_1.fit_transform(dataset["house_ownership"])
print(ds["house_ownership"])
print(le_1.classes_)'''

In [ ]:

#no=0 yes=1 
'''le_2=LabelEncoder()
ds["car_ownership"]=le_2.fit_transform(dataset["car_ownership"])
print(ds["car_ownership"])
print(le_2.classes_)'''

In [ ]:
le_3=LabelEncoder()
ds["profession"]=le_3.fit_transform(dataset["profession"])
print(ds["profession"])
print(le_3.classes_)
print(len(le_3.classes_))

In [ ]:
le_4=LabelEncoder()
ds["city"]=le_4.fit_transform(dataset["city"])
print(ds["city"])
print(le_4.classes_)
print(len(le_4.classes_))

In [ ]:
le_5=LabelEncoder()
ds["state"]=le_5.fit_transform(dataset["state"])
print(ds["state"])
print(le_5.classes_)
print(len(le_5.classes_))

In [ ]:
ds


***3)Visualization and analysis***

In [ ]:
count_e=[]
for i in ds["experience"].unique():
  temp=ds[(ds["experience"]==i) & (ds["risk_flag"]==1)]
  count_e.append(len(temp)/len(ds[ds["experience"]==i]))

plt.scatter((ds["experience"].unique()),count_e)
plt.show()

In [ ]:
count_a=[]
for i in ds["age"].unique():
  temp=ds[(ds["age"]==i) & (ds["risk_flag"]==1)]
  count_a.append(len(temp)/len(ds[ds["age"]==i]))

plt.scatter((ds["age"].unique()),count_a)
plt.show()

In [ ]:
plt.scatter(ds[["income"]],ds[["risk_flag"]])
plt.show()

In [ ]:
count=[]
for i in ds["married"].unique():
  temp=ds[(ds["married"]==i) & (ds["risk_flag"]==1)]
  count.append(len(temp)/len(ds[ds["married"]==i]))

plt.scatter(le.inverse_transform(ds["married"].unique()),count)
plt.show()

***4)Normalization***

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
ds_normalize=pd.DataFrame(scaler.fit_transform(ds.iloc[:,1:-1]))
ds_normalize

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test, y_train, y_test = train_test_split(ds_normalize.iloc[:,:].values,ds.iloc[:,-1:].values, test_size=0.02, random_state=42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

***5)Training***

In [ ]:
from sklearn.ensemble import RandomForestRegressor
n_estimators = 500
max_depth = 25
min_samples_split=2
min_samples_leaf=4
random_forest = RandomForestRegressor(random_state = 42,n_estimators = n_estimators, max_depth = max_depth, min_samples_split=min_samples_split)
random_forest.fit(x_train,y_train.ravel())

In [ ]:
# Decision Tree Regressor
from sklearn.tree import DecisionTreeRegressor
#n_estimators = 100
max_depth = 25
min_samples_split=2
min_samples_leaf=4
decision_tree = DecisionTreeRegressor(max_depth = max_depth, min_samples_split=min_samples_split)
decision_tree.fit(x_train, y_train)

***6)Prediction***

In [ ]:
y_pred_rf= random_forest.predict(x_test)
y_pred_rf
pd.DataFrame(y_pred_rf).to_csv("refer_rf.csv")

In [ ]:
y_pred_dt= decision_tree.predict(x_test)
y_pred_dt
pd.DataFrame(y_pred_dt).to_csv("refer_dt.csv")

In [ ]:
def classif(n):
  if(n>0.05):
    return 1
  return 0

output=list(map(classif,y_pred_rf))
output_dt=list(map(classif,y_pred_dt))
print(output,end=" ")

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, output)

In [ ]:
print(output_dt,end=" ")

### ***ROC_AUC_SCORE***

In [ ]:
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_test, output)
print('ROC AUC_RF: %f' % auc)


In [ ]:
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_test, output_dt)
print('ROC AUC: %f' % auc)

***TESTING***

In [ ]:
testset = pd.read_csv('/content/Test Data.csv')
testset

In [ ]:
testset=testset.drop(["car_ownership","married","house_ownership","current_job_years","current_house_years"], axis=1)
testset

In [ ]:
def conv(string):
  return string.replace(' ', '_')
#testset["married"]=le.transform(testset["married"])
#testset["house_ownership"]=le_1.transform(testset["house_ownership"])
#testset["car_ownership"]=le_2.transform(testset["car_ownership"])
testset["profession"]=list(map(conv,testset["profession"]))
testset["profession"]=le_3.transform(testset["profession"])
testset["city"]=list(map(conv,testset["city"]))
testset["city"]=le_4.transform(testset["city"])
testset["state"]=list(map(conv,testset["state"]))
testset["state"]=le_5.transform(testset["state"])

In [ ]:
testset=testset.iloc[:,1:]

In [ ]:
testset=pd.DataFrame(scaler.transform(testset))

In [ ]:
testset

In [ ]:
y_pred_final= random_forest.predict(testset)
y_pred_final=list(map(classif,y_pred_final))
print(y_pred_final,end=" ")

Selecting Random forest because of the roc_auc score


In [ ]:
pd.DataFrame(y_pred_final).to_csv("Predictions_DataSet.csv")